# Analyze a cross-correlation to produce station clock correction file

Prior to using this script, the quality of the correction should be visualized and confirmed using notebook `plotStationPairXcorr.ipynb`

This notebook will generate a csv file with dates and estimated clock corrections for a given station. Applying the correction to the original ASDF database will be done separately for the sake of safety, so that any changes to ASDF must be very deliberate and intentional.

In [ ]:
import os
import sys
import datetime

In [ ]:
import numpy as np
import scipy
import matplotlib.dates
import matplotlib.pyplot as plt
from dateutil import rrule
import pandas as pd
from scipy.interpolate import UnivariateSpline

In [ ]:
import obspy

In [ ]:
package_root = os.path.abspath('../../..')
if package_root not in sys.path:
    sys.path.append(package_root)
from seismic.xcorqc.xcorr_station_clock_analysis import (XcorrClockAnalyzer, 
                                                         plot_estimated_timeshift)
from seismic.xcorqc.analytic_plot_utils import timestamps_to_plottable_datetimes

In [ ]:
SRC_FILE = "/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/xcorr/AU/HTT_STKA/test/AU.HTT.AU.STKA.nc"
# SRC_FILE = "/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/xcorr/7X/MA43_QIS/7X.MA43.AU.QIS.1.0-10.0.nc"
# SRC_FILE = "/g/data/ha3/Passive/SHARED_DATE/GPS_Clock/xcorr/7X/MA52_QIS/7X.MA52.AU.QIS.1.0-10.0.nc"
assert os.path.exists(SRC_FILE), "File not found!"

In [ ]:
_, basename = os.path.split(SRC_FILE)
_, file_type = os.path.splitext(SRC_FILE)
name_parts = basename.split('.')
NETCODE = name_parts[0]
STATCODE = name_parts[1]
print("Inferred target station code: {}.{}".format(NETCODE, STATCODE))
FULL_CODE = '.'.join([NETCODE, STATCODE])

## Load file

In [ ]:
TIME_WINDOW = 300 # +/-
SNR_THRESHOLD = 6
PCF_CUTOFF_THRESHOLD = 0.5

xcorr_preproc = XcorrClockAnalyzer(SRC_FILE, TIME_WINDOW, SNR_THRESHOLD, PCF_CUTOFF_THRESHOLD)
# flt_times is used for regression
flt_times, correction = xcorr_preproc.get_corrections_time_series()

# plt_times is used for plotting with human readable dates on the x-axis
plt_times = timestamps_to_plottable_datetimes(flt_times)

## Segment the corrections time series into coherent groups

In [ ]:
times_nonan = xcorr_preproc.correction_times_clean
plt_times_nonan = timestamps_to_plottable_datetimes(times_nonan)
correction_nonan = xcorr_preproc.corrections_clean
slope = grad_med5 = xcorr_preproc.corrections_slope

### Perform clustering

In [ ]:
tuned_coeffs = {
    '7X.MA43': (1, 1, 20),
    '7X.MA52': (1, 5, 15),
    '7D.DA41A': (0.4, 0.3, 0.0),
    'AU.HTT': (1, 1, 0)
}

assert FULL_CODE in tuned_coeffs, "Add new coefficients for {}, then manually tune for fit".FULL_CODE

In [ ]:
ind, ids = xcorr_preproc.do_clustering(tuned_coeffs[FULL_CODE])
num_segments = len(set(ids[ids != -1]))
print("{} clusters identified".format(num_segments))

## Plot clusters based on sample positions in time

In [ ]:
plt.figure(figsize=(16,9))

plt.plot(plt_times_nonan, correction_nonan, 'x', color="#808080")
for i in range(max(ids) + 1):
    mask_group = (ids == i)
    plt.plot(plt_times_nonan[mask_group], correction_nonan[mask_group], 'o-', color='C{}'.format(i),
             markersize=6, fillstyle='none', alpha=0.7)

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} first order corrections groups".format(FULL_CODE), fontsize=20)
# ylims = plt.ylim()
plt.show()

## With successful segmentation, we perform regression for each cluster

In [ ]:
# Corrections from regression function fit
correction_fit = np.zeros_like(correction_nonan)
correction_fit[:] = np.nan

In [ ]:
# Fitting univariate spline
degree = [1]*num_segments
regressions = xcorr_preproc.do_spline_regression(ids, degree)
for i in range(num_segments):
    mask_group = (ids == i)
    # Apply regression
    x = times_nonan[mask_group]
    # Compute fitted values
    correction_fit[mask_group] = regressions[i](x)

### Replot with fitted line

In [ ]:
plt.figure(figsize=(16,9))

for i in range(num_segments):
    mask_group = (ids == i)
    plt.plot(plt_times_nonan[mask_group], correction_nonan[mask_group], 'o', color='C{}'.format(i),
             markersize=5, fillstyle='none')
    plt.plot(plt_times_nonan[mask_group], correction_fit[mask_group], color='C{}'.format(i))

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
# plt.ylim(ylims)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.title("Station {} corrections groups with regressions to sample times".format(FULL_CODE), fontsize=20)
plt.show()

## Resample regression lines to the original sample times

In [ ]:
# Dict of daily spaced time values and computed correction, since source data time
# points might not be uniformly distributed. Keyed by group ID. These are the times
# at which we will output corrections.
regular_corrections = {}

In [ ]:
sec_per_day = 24*3600

for i in range(num_segments):
    mask_group = (ids == i)
    # Generate uniform daily times at which to compute corrections
    x = times_nonan[mask_group]
    timestamp_min = min(x)
    timestamp_max = max(x)
    num_days = np.round((timestamp_max - timestamp_min)/sec_per_day)
    lin_times = np.linspace(timestamp_min, timestamp_max, num_days + 1)
    lin_corrections = regressions[i](lin_times)
    regular_corrections[i] = {'times': lin_times, 'corrections': lin_corrections}

In [ ]:
# Replot to sanity check the final daily correction values
plt.figure(figsize=(16,9))

for i in range(num_segments):
    mask_group = (ids == i)
    plt.plot(plt_times_nonan[mask_group], correction_nonan[mask_group], 'o', color='#808080'.format(i),
             markersize=5, fillstyle='none')
    plt_times_i = np.array([datetime.datetime.utcfromtimestamp(v) for v in 
                            regular_corrections[i]['times']]).astype('datetime64[ms]')
    plt.plot(plt_times_i, regular_corrections[i]['corrections'], 'o', color='C{}'.format(i), fillstyle='none')

time_formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
plt.axes().xaxis.set_major_formatter(time_formatter)
# plt.ylim(ylims)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(':', color="#808080", zorder=0, alpha=0.5)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Correction (sec)', fontsize=14)
plt.gcf().autofmt_xdate()
plt.title("Station {} corrections groups with regressions to daily samples".format(FULL_CODE), fontsize=20)
plt.tight_layout()
plt.savefig(FULL_CODE + "_clock_correction_profile.png", dpi=300)
plt.show()

## Output regression results to csv file

Use tabular format for ease of use and interoperability, even though there will be some redundancy of information.

In [ ]:
data_blocks = []
for k in regular_corrections.keys():
    c = regular_corrections[k]
    # BEWARE: The 'corrections' array sign is negated there, since the correction
    # we have computed up to this point is actually the clock *error*. Subtraction
    # of an error is the same as addition of a correction of opposite sign.
    data_blocks.append(pd.DataFrame(np.column_stack([c['times'], -c['corrections']]), 
                                    columns=['timestamp', 'clock_correction']))
df = pd.concat(data_blocks)

In [ ]:
df['date'] = df['timestamp'].apply(obspy.UTCDateTime).apply(lambda x: x.date)
df['net'] = NETCODE
df['sta'] = STATCODE
df = df[['net', 'sta', 'date', 'clock_correction']]

In [ ]:
output_file = FULL_CODE + "_clock_correction.csv"

In [ ]:
print("Exporting corrections to file {}".format(output_file))
df.to_csv(output_file, index=False)